In [1]:
import os
import pandas as pd
import numpy as np
import librosa
import librosa.display
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from sklearn.model_selection import train_test_split
from tqdm import tqdm 

/tmp/ipykernel_7189/1305221833.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
2025-03-12 21:06:43.027339: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-12 21:06:44.134545: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-12 21:06:44.624665: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741

In [2]:
# ----------- 1️. Load CSV and Extract Features -----------
data_path = "Audio"
csv_path = "updated_dataset.csv"  

# Load dataset
df = pd.read_csv(csv_path)  

In [3]:
df['age'].fillna(45.0,inplace=True)

/tmp/ipykernel_7189/4252838758.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['age'].fillna(45.0,inplace=True)


In [4]:
# Audio Processing Function
def extract_mel_spectrogram(file_path, sr=16000, duration=3, n_mels=128):
    try:
        y, sr = librosa.load(file_path, sr=sr, duration=duration)  # Load WAV/MP3
        mel_spec = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=n_mels)
        mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max)  # Convert to dB
        return mel_spec_db
    except Exception as e:
        print(f"Error loading {file_path}: {e}")
        return None

In [5]:
# Process dataset
X, y = [], []
for _, row in tqdm(df.iterrows(), total=len(df)): 
    filename, age = row['filename'], row['age']
    file_path = os.path.join(data_path, filename)

    # Check if file exists
    if os.path.exists(file_path) and filename.endswith(('.wav', '.mp3')):
        mel_spec = extract_mel_spectrogram(file_path)
        
        if mel_spec is not None:
            if mel_spec.shape != (128, 128):  # Resize if necessary
                mel_spec = np.resize(mel_spec, (128, 128))
            
            X.append(mel_spec)
            y.append(age)


100%|███████████████████████████████████████| 1482/1482 [00:33<00:00, 44.04it/s]


In [6]:
X = np.array(X).reshape(-1, 128, 128, 1)  # Add channel dimension
y = np.array(y)

In [7]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
# ----------- 2️.Build CNN Model -----------
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 1)),
    BatchNormalization(),
    MaxPooling2D((2, 2)),

    Conv2D(64, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),

    Conv2D(128, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dense(1)  # Output layer for regression (age prediction)
])


/home/ravindra/.local/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2025-03-12 21:09:08.180653: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [9]:
# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

In [10]:

# ----------- 3️. Train the Model -----------
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=20, batch_size=32)

Epoch 1/20


2025-03-12 21:09:20.374210: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 77660160 exceeds 10% of free system memory.
2025-03-12 21:09:25.155261: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 65028096 exceeds 10% of free system memory.
2025-03-12 21:09:25.556012: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 65028096 exceeds 10% of free system memory.
2025-03-12 21:09:25.663834: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 65028096 exceeds 10% of free system memory.
2025-03-12 21:09:25.710277: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 16257024 exceeds 10% of free system memory.


38/38 ━━━━━━━━━━━━━━━━━━━━ 59s 1s/step - loss: 600.5993 - mae: 19.0691 - val_loss: 192.9968 - val_mae: 11.2662
Epoch 2/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 53s 1s/step - loss: 215.6123 - mae: 11.8077 - val_loss: 715.8430 - val_mae: 23.2956
Epoch 3/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 54s 1s/step - loss: 169.6237 - mae: 10.5063 - val_loss: 1043.9834 - val_mae: 29.5242
Epoch 4/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 54s 1s/step - loss: 165.7137 - mae: 10.4640 - val_loss: 589.8318 - val_mae: 20.5217
Epoch 5/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 56s 1s/step - loss: 140.0744 - mae: 9.4292 - val_loss: 672.1478 - val_mae: 22.0688
Epoch 6/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 54s 1s/step - loss: 122.9875 - mae: 8.8854 - val_loss: 568.8351 - val_mae: 19.8949
Epoch 7/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 55s 1s/step - loss: 102.5631 - mae: 8.0442 - val_loss: 475.3660 - val_mae: 17.9123
Epoch 8/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 54s 1s/step - loss: 80.3122 - mae: 7.1043 - val_loss: 734.7770 - val_mae: 23.7846
Epoch 9/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 5

In [11]:
# Evaluate the model
loss, mae = model.evaluate(X_test, y_test)
print(f"Test MAE: {mae:.2f} years")

10/10 ━━━━━━━━━━━━━━━━━━━━ 3s 281ms/step - loss: 215.6835 - mae: 11.8407
Test MAE: 11.45 years


In [12]:
# Save the model
model.save("age_prediction_model.h5")

In [ ]:
import os
import numpy as np
import pandas as pd
import librosa
import tensorflow as tf

# Load trained model
model = tf.keras.models.load_model("age_prediction_model.h5")

# Function to extract Mel-Spectrogram
def extract_mel_spectrogram(file_path, sr=16000, duration=3, n_mels=128):
    try:
        y, sr = librosa.load(file_path, sr=sr, duration=duration)  # Load WAV/MP3
        mel_spec = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=n_mels)
        mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max)  # Convert to dB

        # Normalize spectrogram
        mel_spec_db = (mel_spec_db - np.min(mel_spec_db)) / (np.max(mel_spec_db) - np.min(mel_spec_db))

        # Resize to match model input
        mel_spec_db = np.resize(mel_spec_db, (128, 128))
        
        return mel_spec_db
    except Exception as e:
        print(f"Error processing {file_path}: {e}")
        return None

# Function to predict age from a CSV file
def predict_ages(csv_file, audio_folder):
    df = pd.read_csv(csv_file)  # Load CSV
    df['age'].fillna(45.0,inplace=True)
    predictions = []
    
    for _, row in df.iterrows():
        filename = row['filename']
        file_path = os.path.join(audio_folder, filename)

        if os.path.exists(file_path) and filename.endswith(('.wav', '.mp3')):
            mel_spec = extract_mel_spectrogram(file_path)
            
            if mel_spec is not None:
                mel_spec = np.array(mel_spec).reshape(1, 128, 128, 1)  # Reshape for model
                predicted_age = model.predict(mel_spec)[0][0]  # Get prediction
                predictions.append((filename, predicted_age))

    # Convert predictions to DataFrame
    predictions_df = pd.DataFrame(predictions, columns=['filename', 'predicted_age'])

    # Find individuals whose age is greater than 60
    seniors_df = predictions_df[predictions_df['predicted_age'] > 60]

    print("\n🔹 **Predicted Ages:**")
    print(predictions_df)

    print("\n🚨 **Individuals Older Than 60:**")
    print(seniors_df)

    return predictions_df, seniors_df

# Example usage
csv_path = "updated_dataset.csv"  # CSV containing filenames
audio_folder = "Audio"  # Folder containing .mp3 and .wav files

predicted_ages, seniors = predict_ages(csv_path, audio_folder)


/tmp/ipykernel_7189/3763318887.py:31: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['age'].fillna(45.0,inplace=True)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 218ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━